# lec 4 Use SQL

## review python

In [1]:
demo_str = 'this is my string'

for word_item in demo_str.split():
    print (word_item)

this
is
my
string


In [2]:
print(' {}+{} is {} '.format(1, 2, 1+2))

 1+2 is 3 
